# Example using Dynamic Frienemy Prunning (DFP)

In this example we show how to apply an online prunning algorithm which can improve the classification accuracy for DS techniques.

The DFP method is an alternative to deal with heavily imbalanced problems, as it prunes the base classifiers that are biased to the majority class.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

# Example of DCS techniques
from DCS.OLA import OLA
from DCS.APriori import APriori
# Example of DES techniques
from DES.KNORAE import KNORAE
from DES.DESP import DESP

## Loading a classification dataset and preparing the data

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

# split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Split the data into training and DSEL for DS techniques
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train, test_size=0.5)

## Training a pool of classifiers

Here we generate a pool of classifiers using the Bagging technique. The Perceptron classifier is used as the base classifier model in this example. It is important to mention that some DS techniques requires that the base classifiers are capable of estimating probabilities. For the Perceptron model, this can be achieved by calibrating the outputs of the base classifiers using the CalibratedClassifierCV class from scikit-learn.

In [ ]:
# Calibrating Perceptrons to estimate probabilities
model = CalibratedClassifierCV(Perceptron(), cv=3, method='sigmoid')
pool_classifiers = BaggingClassifier(n_estimators=10)
pool_classifiers.fit(X_train, y_train)

## Initializing DS techniques

Here we initialize the DS techniques. Three DCS and three DES techniques are considered in this example. The only parameter that is required by the techniques is the pool of classifiers. All others are optional parameters which can be specified.

In [ ]:
# DS techniques without DFP
knorae = KNORAE(pool_classifiers)
ola = OLA(pool_classifiers)
apriori = APriori(pool_classifiers)
desp = DESP(pool_classifiers)

# DS techniques using DFP
fire_knorae = KNORAE(pool_classifiers, DFP=True)
fire_ola = OLA(pool_classifiers, DFP=True)
fire_apriori = APriori(pool_classifiers, DFP=True)
fire_desp = DESP(pool_classifiers, DFP=True)

Each DS technique implements the basic routines from the scikit-learning (fit, predict, predict_proba and score). Here, we call the function fit to prepare the DS techniques for the classification of new data. In this case, the function fit pre-process the information required to apply the DS techniques such as the method that is used to estimate the region of competence.

In [ ]:
knorae.fit(X_dsel, y_dsel)
ola.fit(X_dsel, y_dsel)
apriori.fit(X_dsel, y_dsel)
desp.fit(X_dsel, y_dsel)

fire_knorae.fit(X_dsel, y_dsel)
fire_ola.fit(X_dsel, y_dsel)
fire_apriori.fit(X_dsel, y_dsel)
fire_desp.fit(X_dsel, y_dsel)

## Calculate classification accuracy of each technique

Each DS technique implements the function score from scikit-learn in order to estimate the classification accuracy.

In [ ]:
print('Classification accuracy of KNORAE: ', knorae.score(X_test, y_test))
print('Classification accuracy of Fire-KNORAE: ', fire_knorae.score(X_test, y_test))

print('Classification accuracy of OLA: ', ola.score(X_test, y_test))
print('Classification accuracy of Fire-OLA: ', fire_ola.score(X_test, y_test))

print('Classification accuracy of A priori: ', apriori.score(X_test, y_test))
print('Classification accuracy of Fire-A priori: ', fire_apriori.score(X_test, y_test))

print('Classification accuracy of DES-P: ', desp.score(X_test, y_test))
print('Classification accuracy of Fire-DES-P: ', fire_desp.score(X_test, y_test))